In [39]:
#IMPORT BIBLIOTEK
import sympy as sp
import katalog
import importlib
from IPython.display import display, Math
#przeładowanie modułu importu z katalog
importlib.reload(katalog)
#wyswieltanie wzorow matematycznych
sp.init_printing()

### ZALEŻNOŚCI GEOMETRYCZNE I ZMIENNE WEJŚCIOWE

Poniżej zestawienie zmiennych wykorzystywanych w modelu. Dane geometryczne profili UPE pobierane są automatycznie z bazy (`katalog.py`) na podstawie normy DIN 1026-2.

#### Parametry wejściowe (z Katalogu UPE):
* $h_c$ - wysokość ceownika [mm] (wymiar wzdłuż osi Z w układzie globalnym)
* $b_c$ - szerokość stopki ceownika [mm] (wymiar wzdłuż osi Y w układzie globalnym)
* $t_{wc}$ - grubość środnika [mm]
* $t_{fc}$ - grubość stopki [mm]
* $r_c$ - promień zaokrąglenia naroża [mm]
* $A_c$ - pole przekroju poprzecznego jednego ceownika [mm²]
* $x_c$ - odległość środka ciężkości od zewnętrznej krawędzi środnika [mm]
* $I_{cy}$ - moment bezwładności ceownika względem własnej osi mocnej [mm⁴]
* $I_{cz}$ - moment bezwładności ceownika względem własnej osi słabej [mm⁴]
* $E_{def}$ - wartość współczynika wg Eurokod 4 dla określonego materiału ceownika,
* $R_{e}$ - granica plastyczności ceownika [MPa] 

#### Parametry płaskownika:
* $b_p$ - szerokość płaskownika [mm]
* $t_p$ - grubość płaskownika [mm]

#### Zmienne obliczeniowe konstrukcji:
Wzory definiujące geometrię złożoną:

* $b_{cal} = b_p$ - Szerokość konstrukcji jest równa szerokości płaskownika
* $h_{cal} = h_c + t_p$ - Wysokość całkowita to suma szerokości ceownika (stojącego pionowo) i grubości płaskownika
* $b_{otw} = b_{cal} - 2*b_c$ - Szerokość otwarcia - szerokość płaskownika pomniejszona o podwójną wysokość ceowników

#### Wymiary obliczeniowe (wg rysunku), pomiędzy osiami ścianek a osiami głównymi:

* $y_{cal} = h_c + t_p/2 - t_{fc}/2$ - odległość od OZ (osi płaskownika) do osi przeciwległej (dolnej) stopki (na rysunku jako $y_f$),
* $y_f = h_c - t_{fc}$ - odległość pomiędzy osiami stopek ceownika,
* $y_e = t_p/2 + t_{fc}/2$ - odległość pomiędzy osią płaskownika a osią przyległej (górnej) stopki ceownika (na rysunku jako $y_e$),
* $z_c = b_p/2 - t_{wc}/2$ - odległość pomiędzy osią środnika a OY (połowa rozstawu osiowego),
* $z_f = b_c - t_{wc}/2$ - odległość od końca stopki ceownika do osi środnika (na rysunku wymiar poziomy stopki),
* $z_{cc} = x_c - t_{wc}/2$ - odległość od osi środnika do środka ciężkości ceownika (na rysunku $z_{cc}$),
* $z_p = 2*z_c$ - odległość pomiędzy osiami środników ceowników, czyli szerokość obliczeniowa płaskownika.

### Definicja układu współrzędnych
Zgodnie z rysunkiem pomocniczym:
* $X$ - czyli oś długa pręta,
* $Y$ - czyli oś pionowa przechodząca przez jedyny środek symetrii profilu (skierowana w dół),
* $Z$ - czyli oś przechodząca przez środek płaskownika (nie pokrywa się z tylną ścianką profilu),
* $Z_{c}$ - oś równoległa do Z przechodząca przez środek ciężkości $S_c$ profilu,
* $Z_{s}$ - oś równoległa do Z przechodząca przez środek ścinania $S_s$ profilu,
* $Z_{0}$ - oś równoległa do Z przechodząca przez tylną ściankę płaskownika,
* $y_c$ - odległość od osi Y do środka ciężkości $S_c$ profilu,
* $y_s$ - odległość od osi Y do środka ścinania $S_s$ profilu.

In [40]:
#ZALEŻNOŚCI GEOMETRYCZNE I ZMIENNE WEJŚCIOWE
hc, bc, twc, tfc, rc = sp.symbols('hc bc twc tfc rc', real=True, positive=True)
Ac, xc, Icy, Icz = sp.symbols('Ac xc Icy Icz', real=True, positive=True)
bp, tp = sp.symbols('bp tp', real=True, positive=True)
yc, ys = sp.symbols('yc ys', real=True)

# ZMIENNE KONSTRUKCYJNE
b_cal = bp
h_cal = hc + tp
b_otw = b_cal - 2 * hc

# WYMIARY OBLICZENIOWE (OSIOWE)
# odległość od OZ do osi dolnej stopki
y_cal = hc + tp/2 - tfc/2
# rozstaw osi stopek ceownika
y_f = hc - tfc
# odległość od OZ do osi górnej stopki
y_e = tp/2 + tfc/2
# odległość od OY do osi środnika
z_c = bp/2 - twc/2
# długość stopki (od osi środnika)
z_f = bc - twc/2
# odległość osi środnika do SC ceownika
z_cc = xc - twc/2
# rozstaw osi środników (szerokość obliczeniowa płaskownika)
z_p = 2 * z_c

### KLASYFIKACJA PRZEKROJU, POLA POWIERZCHNI, ŚRODEK CIĘŻKOŚCI ($Y_C$), MOMENTY BEZWŁADNOŚCI PROFILU, PROMIENIE BEZWŁADNOŚCI, WSKAŹNIKI WYTRZYMAŁOŚCI NA ZGINANIE

### Klasyfikacja przekroju wg normy Eurokod 3 (PN-EN 1993-1-1)
*Założenie: Płaskownik (góra) jest rozciągany, końcówki stopek ceowników (dół) są ściskane.*

Parametr materiałowy:
$$\epsilon = \sqrt{\frac{E_{def}}{R_{ec}}}$$
*(gdzie $R_{ec}$ to granica plastyczności stali w MPa)*

#### Środnik ceownika (częściowo ściskany)
Środnik jest elementem przęsłowym (podpartym na obu końcach).
Wysokość obliczeniowa środnika (odcinek płaski):
$$c_w = h_c - 2 \cdot t_{fc} - 2 \cdot r_c$$

Klasyfikacja w zależności od rozkładu naprężeń ($\alpha$):
* Klasa 1: $\frac{c_w}{t_{wc}} \le \frac{396 \cdot \epsilon}{13 \cdot \alpha - 1}$
* Klasa 2: $\frac{c_w}{t_{wc}} \le \frac{456 \cdot \epsilon}{13 \cdot \alpha - 1}$

**Współczynnik $\alpha$ (udział strefy ściskanej):**
Określa, jaka część środnika znajduje się poniżej osi obojętnej (w strefie ściskania).
$$\alpha = \frac{(t_p/2 + h_c - t_{fc}) - y_c}{c_w}$$

#### Stopka ceownika (ściskana)
Dolna stopka ceownika jest elementem wspornikowym (wolny koniec), ściskanym równomiernie.
Wysięg stopki: $c_f = b_c - t_{wc} - r_c$

* Klasa 1: $\frac{c_f}{t_{fc}} \le 9 \cdot \epsilon$
* Klasa 2: $\frac{c_f}{t_{fc}} \le 10 \cdot \epsilon$
* Klasa 3: $\frac{c_f}{t_{fc}} \le 14 \cdot \epsilon$


### Pola powierzchni
$$A_p = b_p \cdot t_p \quad [mm^2]$$
$$A_{cal} = 2 \cdot A_c + A_p \quad [mm^2]$$

### Środek ciężkości ($y_c$)
Odległość od osi $OZ$ (przechodzącej przez środek ciężkości płaskownika) do globalnego środka ciężkości $S_c$.

$$y_c = \frac{2 \cdot A_c \cdot (t_p/2 + h_c/2)}{A_{cal}} \quad [mm]$$

### Momenty bezwładności profilu

#### Względem osi głównej $OZ_c$ (Zginanie pionowe)
* Ceowniki pracują osią **MOCNĄ** ($I_{cy}$), ponieważ stoją pionowo.
* Ramię Steinera to odległość pionowa pomiędzy środkiem ciężkości ceownika i całego profilu.

$$I_{zc} = \left(\frac{b_p \cdot t_p^3}{12} + A_p \cdot y_c^2\right) + 2 \cdot \left( I_{cy} + A_c \cdot \left(\frac{t_p}{2} + \frac{h_c}{2} - y_c\right)^2 \right) \quad [mm^4]$$

#### Względem osi symetrii $OY$ (Zginanie boczne)
* Ceowniki pracują osią **SŁABĄ** ($I_{cz}$), ponieważ zginanie następuje w bok.
* Ramię Steinera to odległość pozioma środków ciężkości ($z_c - z_{cc}$).

$$I_y = \frac{t_p \cdot b_p^3}{12} + 2 \cdot \left( I_{cz} + A_c \cdot (z_c - z_{cc})^2 \right) \quad [mm^4]$$

---

### Promienie bezwładności
$$i_y = \sqrt{\frac{I_y}{A_{cal}}} \quad [mm]$$
$$i_z = \sqrt{\frac{I_{zc}}{A_{cal}}} \quad [mm]$$

---

### Wskaźniki wytrzymałości na zginanie
Minimalne wskaźniki dla najbardziej wytężonych włókien.

* **Względem osi $OZ_c$ ($W_z$) - Zginanie góra-dół:**
Skrajnym włóknem jest dolna krawędź stopki ceownika.
$$y_{max} = (t_p/2 + h_c) - y_c$$
$$W_z = \frac{I_{zc}}{y_{max}} \quad [mm^3]$$

* **Względem osi $OY$ ($W_y$) - Zginanie boczne:**
Skrajnym włóknem jest boczna krawędź płaskownika (szerokość $b_p$).
$$W_y = \frac{I_y}{b_p/2} \quad [mm^3]$$

In [41]:
# KLASYFIKACJA PRZEKROJU, POLA POWIERZCHNI, ŚRODEK CIĘŻKOŚCI (yc), MOMENTY BEZWŁADNOŚCI PROFILU, PROMIENIE BEZWŁADNOŚCI, WSKAŹNIKI WYTRZYMAŁOŚCI NA ZGINANIE
# Re - granica plastyczności, Edef - współczynnik odniesienia (np. 235)
Re, Edef = sp.symbols('Re Edef', real=True, positive=True)
epsilon = sp.sqrt(Edef / Re)

# POLA POWIERZCHNI
Ap = bp * tp
Acal = 2 * Ac + Ap

# ŚRODEK CIĘŻKOŚCI (yc) od osi OZ w dół
yc = (2 * Ac * (tp/2 + hc/2)) / Acal

# KLASYFIKACJA PRZEKROJU WG EUROKOD 3

# Środnik ceownika (częściowo ściskany)
## Wysokość obliczeniowa środnika
cw = hc - 2*tfc - 2*rc

## Współczynnik alpha (udział strefy ściskanej)
alpha = ((tp/2 + hc - tfc) - yc) / cw

## Limity smukłości dla środnika (Klasa 1 i 2)
limit_web_c1 = (396 * epsilon) / (13 * alpha - 1)
limit_web_c2 = (456 * epsilon) / (13 * alpha - 1)
smuklosc_web = cw / twc

# Stopka ceownika (ściskana)
## Wysięg stopki
cf = bc - twc - rc

## Limity smukłości dla stopki (Klasa 1, 2, 3)
limit_flange_c1 = 9 * epsilon
limit_flange_c2 = 10 * epsilon
limit_flange_c3 = 14 * epsilon
smuklosc_flange = cf / tfc

# MOMENTY BEZWŁADNOŚCI PROFILU

# Względem osi głównej OZc (Zginanie pionowe - oś mocna ceowników)
# Steiner: odległość pionowa środków ciężkości
Izc = (bp * tp**3)/12 + Ap * yc**2 + 2 * (Icy + Ac * (tp/2 + hc/2 - yc)**2)

# Względem osi symetrii OY (Zginanie boczne - oś słaba ceowników)
# Steiner: odległość pozioma środków ciężkości (zc i zcc zdefiniowane wcześniej)
Iy = (tp * bp**3)/12 + 2 * (Icz + Ac * (z_c - z_cc)**2)

# PROMIENIE BEZWŁADNOŚCI
iy = sp.sqrt(Iy / Acal)
iz = sp.sqrt(Izc / Acal)

# WSKAŹNIKI WYTRZYMAŁOŚCI NA ZGINANIE

# Względem osi OZc (Wz) - włókno dolne
ymax = (tp/2 + hc) - yc
Wz = Izc / ymax

# Względem osi OY (Wy) - krawędź boczna
Wy = Iy / (bp/2)

## Wyznaczenie Środka Ścinania ($S_s$) - Teoria Własowa

Dla profili cienkościennych o jednej osi symetrii ($OY$), środek ścinania leży na tej osi, jednak jego rzędna pionowa ($y_s$) jest przesunięta względem środka ciężkości ($y_c$). Aby wyznaczyć to przesunięcie $\Delta y_s$, stosujemy **Teorię Własowa** w ujęciu wycinkowym.

### Definicja fizyczna
Środek ścinania to punkt, w którym przyłożenie siły poprzecznej (np. $V_z$) nie powoduje skręcania profilu. Warunkiem równowagi jest, aby moment od siły zewnętrznej był równoważony przez moment od wewnętrznych sił stycznych (przepływu ścinania) w przekroju.

Równanie równowagi momentów względem bieguna w Środku Ciężkości ($S_c$):
$$V_z \cdot (y_s - y_c) = \int_{A} q(s) \cdot r(s) \cdot ds$$

Gdzie:
* $V_z$ – siła tnąca pozioma,
* $(y_s - y_c)$ – szukane ramię siły zewnętrznej (przesunięcie $\Delta y_s$),
* $q(s)$ – przepływ ścinania [$N/mm$],
* $r(s)$ – odległość stycznej do konturu ścianki od bieguna $S_c$.

### Przejście na współrzędne wycinkowe ($\omega$)
Wykorzystując wzór Żurawskiego na przepływ ścinania ($q = \frac{V_z S_y}{I_y}$) i wprowadzając definicję **współrzędnej wycinkowej** ($d\omega = r \cdot ds$), otrzymujemy finalny wzór całkowy Własowa:

$$\Delta y_s = y_s - y_c = \frac{1}{I_y} \int_{A} \omega_{Sc}(s) \cdot z_{glob}(s) \cdot dA$$

Gdzie:
* $\omega_{Sc}(s)$ – współrzędna wycinkowa liczona względem bieguna $S_c$ (pole omiatane przez promień wodzący),
* $z_{glob}(s)$ – globalna współrzędna pozioma punktu na profilu (ramię dla momentu bezwładności $I_y$),
* $dA = t \cdot ds$ – różniczkowe pole przekroju.

---

### Algorytm obliczeń dla profilu złożonego (Metoda 4 Stref)

Ze względu na skokowe zmiany grubości ścianek i geometrii (płaskownik + ceownik), całkę $\int \omega \cdot z \cdot dA$ (zwaną wycinkowym momentem statycznym) obliczamy jako sumę czteReh niezależnych etapów. Całkowanie rozpoczynamy w osi symetrii ($z_{glob}=0, \omega=0$).

#### Etap 1: Czysty Płaskownik
*Odcinek poziomy od osi symetrii do krawędzi wewnętrznej stopki.*
* **Ruch:** Wzdłuż osi $Z$ (poziomo).
* **Ramię wycinka:** Stała odległość pionowa $r = y_{plask\_r}$ (odległość $S_c$ od osi płaskownika).
* **Funkcja $\omega$:** Rośnie liniowo.
$$\omega_1(s) = y_{plask\_r} \cdot s$$
* **Całka:** Globalna współrzędna $z_{glob}$ jest równa lokalnej drodze $s$.
$$S_{\omega,1} = \int \omega_1(s) \cdot s \cdot t_p \, ds$$

#### Etap 2: Strefa Zakładki (Płaskownik + Górna Stopka)
*Odcinek poziomy, gdzie płaskownik leży na stopce ceownika.*
* **Ruch:** Kontynuacja wzdłuż osi $Z$.
* **Ramię wycinka:** Stała odległość pionowa $r = y_{zakladka\_r}$ (skorygowana do środka ciężkości połączenia).
* **Grubość:** Sumaryczna $t = t_p + t_{fc}$.
* **Funkcja:** Startuje od wartości końcowej Etapu 1.
$$\omega_2(s) = \omega_{end,1} + y_{zakladka\_r} \cdot s$$
* **Całka:** Globalna współrzędna to suma drogi z etapu 1 i obecnej: $z_{glob} = L_1 + s$.

#### Etap 3: Środnik Ceownika (Odcinek pionowy)
*Odcinek wzdłuż środnika w dół.*
* **Ruch:** Wzdłuż osi $Y$ (zmiana kierunku o 90° - pionowo).
* **Ramię wycinka:** Stała odległość **pozioma** $r = z_{srodnik\_r}$ (odległość $S_c$ od osi środnika).
* **Funkcja:** Startuje od wartości w górnym narożniku.
$$\omega_3(s) = \omega_{corner,top} + z_{srodnik\_r} \cdot s$$
* **Całka:** Kluczowa uwaga – w całce $\int \omega \cdot z \cdot dA$ wartość $z_{glob}$ jest tutaj **STAŁA** i równa $z_{srodnik\_r}$, ponieważ cały środnik leży w tej samej odległości od osi symetrii $OY$.
$$S_{\omega,3} = \int \omega_3(s) \cdot z_{srodnik\_r} \cdot t_{wc} \, ds$$

#### Etap 4: Dolna Stopka (Powrót)
*Odcinek poziomy od dolnego naroża w stronę osi symetrii.*
* **Ruch:** Wzdłuż osi $Z$ (do wewnątrz profilu).
* **Ramię wycinka:** Stała odległość pionowa $r = y_{stopka\_d\_r}$.
* **Funkcja:** Startuje od wartości w dolnym narożniku.
$$\omega_4(s) = \omega_{corner,bot} + y_{stopka\_d\_r} \cdot s$$
* **Całka:** Globalna współrzędna $z_{glob}$ maleje wraz z postępem całkowania, ponieważ wracamy do osi $OY$.

### Wynik końcowy
Ostateczne przesunięcie środka ścinania jest sumą wkładów ze wszystkich stref (pomnożoną przez 2 dla całego profilu):
$$\Delta y_s = \frac{2 \cdot (S_{\omega,1} + S_{\omega,2} + S_{\omega,3} + S_{\omega,4})}{I_y}$$

In [42]:
# WYZNACZANIE ŚRODKA ŚCINANIA (Współrzędna pionowa Ys)

# Zmienne lokalne do całkowania (zmienna drogi 's' po konturze)
# z_lokalne - zmienna całkowania wzdłuż danego segmentu
s_var = sp.symbols('s_var', real=True)


# PARAMETRY GEOMETRYCZNE (Ramiona sił / współrzędne)
# Odległość pionowa od środka ciężkości (Sc) do osi płaskownika
ramie_pionowe_plaskownik = yc 

# Odległość pionowa od Sc do osi zakładki (wspólna część płaskownika i stopki) oś zakładki leży w połowie sumarycznej grubości (tp + tfc)
ramie_pionowe_zakladka = yc - tfc/2

# Odległość pozioma od Sc do osi środnika (stała odległość 'z' dla całego środnika)
ramie_poziome_srodnik = z_c 

# Odległość pionowa od Sc do osi dolnej stopki 
ramie_pionowe_stopka_dol = h_cal - yc - tfc/2


# Całkowanie po konturze


# ETAP 1: CZYSTY PŁASKOWNIK (od środka do krawędzi stopki) 

# Długość tego odcinka: połowa szerokości płaskownika minus szerokość ceownika
dlugosc_strefy_1 = bp/2 - bc
# Funkcja wycinkowa omega = ramię_pionowe * droga
omega_1 = ramie_pionowe_plaskownik * s_var
# Moment wycinkowy strefy 1 (Sw) całka: omega * (współrzędna_z_globalna) * grubość, zmienna s_var pokrywa się ze współrzędną 'z'
Sw_1 = sp.integrate(omega_1 * s_var * tp, (s_var, 0, dlugosc_strefy_1))
# Wartość omega na końcu strefy 1 (potrzebna do ciągłości funkcji)
omega_koniec_1 = omega_1.subs(s_var, dlugosc_strefy_1)

# ETAP 2: STREFA ZAKŁADKI

# Długość tego odcinka to szerokość stopki górnej ceownika
dlugosc_strefy_2 = bc
# Funkcja omega startuje od wartości końcowej z etapu 1, ramię siły (ramie_pionowe_zakladka) jest tutaj inne niż w etapie 1
omega_2 = omega_koniec_1 + ramie_pionowe_zakladka * s_var 
# Grubość sumaryczna (płaskownik + stopka ceownika)
t_zakladka = tp + tfc
# Moment wycinkowy strefy 2, współrzędna 'z' globalna to suma drogi z etapu 1 i obecnej zmiennej s_var
z_globalne_2 = dlugosc_strefy_1 + s_var
Sw_2 = sp.integrate(omega_2 * z_globalne_2 * t_zakladka, (s_var, 0, dlugosc_strefy_2))
# Wartość omega w górnym narożniku (koniec pasa górnego)
omega_naroznik_gora = omega_2.subs(s_var, dlugosc_strefy_2)

# ETAP 3: ŚRODNIK (Pionowo w dół)

# Długość tego odcinka: odległość między osiami stopek
dlugosc_strefy_3 = y_f
# Funkcja omega na środniku, ramię to odległość pozioma środnika od osi Y (stałe z_c)
omega_3 = omega_naroznik_gora + ramie_poziome_srodnik * s_var
# Moment wycinkowy strefy 3, tutaj globalna współrzędna 'z' jest STAŁA i wynosi ramie_poziome_srodnik (z_c)
Sw_3 = sp.integrate(omega_3 * ramie_poziome_srodnik * twc, (s_var, 0, dlugosc_strefy_3))
# Wartość omega w dolnym narożniku
omega_naroznik_dol = omega_3.subs(s_var, dlugosc_strefy_3)

# ETAP 4: DOLNA STOPKA (Od naroża w lewo)
# Długość efektywna dolnej stopki
dlugosc_strefy_4 = z_f
# Funkcja omega na dolnej stopce
omega_4 = omega_naroznik_dol + ramie_pionowe_stopka_dol * s_var
# Moment wycinkowy strefy 4, globalna współrzędna 'z' maleje, idąc od środnika w lewo (z_c - droga)
z_globalne_4 = z_c - s_var
Sw_4 = sp.integrate(omega_4 * z_globalne_4 * tfc, (s_var, 0, dlugosc_strefy_4))

# WYNIK KOŃCOWY

# Całkowity moment wycinkowy dla całego przekroju mnożymy razy 2, ponieważ obliczenia prowadziliśmy dla połowy profilu (symetria lewa-prawa)
Sw_calkowite = 2 * (Sw_1 + Sw_2 + Sw_3 + Sw_4)

# Przesunięcie środka ścinania względem środka ciężkości (delta Ys) wynika z definicji: Ys = Integral(omega * z * dA) / Iy
delta_ys = Sw_calkowite / Iy

# Globalne położenie środka ścinania zakładając, że oś Y rośnie w dół, odejmujemy przesunięcie od współrzędnej środka ciężkości
ys = yc - delta_ys

#DEFINICJA FUNKCJI OMEGA_SS (Względem Środka Ścinania) Transformacja omega z Sc do Ss

# Wzór transformacyjny: omega_Ss = omega_Sc - delta_ys * z_globalne
omega_ss_1 = (omega_1 - delta_ys * s_var).simplify()
omega_ss_2 = (omega_2 - delta_ys * (dlugosc_strefy_1 + s_var)).simplify()
omega_ss_3 = (omega_3 - delta_ys * ramie_poziome_srodnik).simplify()
omega_ss_4 = (omega_4 - delta_ys * (z_c - s_var)).simplify()

### GŁÓWNA WSPÓŁRZĘDNA WYCINKOWA, WYCINKOWY MOMENT BEZWŁADNOŚCI, BIEGUNOWY MOMENT BEZWŁADNOŚCI (WZGLĘDEM Ss), MOMENT BEZWŁASNOŚCI PRZY SKRĘCANIU SWOBODNYM

Po wyznaczeniu położenia Środka Ścinania ($S_s$), konieczne jest przeliczenie charakterystyk geometrycznych tak, aby odnosiły się one do tego punktu. Jest to niezbędne do dalszej analizy zwichrzenia, ponieważ przy skręcaniu nieswobodnym przekrój "obraca się" właśnie wokół środka ścinania.

#### 1. Główna współrzędna wycinkowa ($\omega_{Ss}$)
Wcześniej wyznaczona funkcja $\omega(s)$ była liczona względem bieguna w środku ciężkości ($S_c$). Taki układ nie jest układem głównym dla skręcania.

Aby uzyskać **główną współrzędną wycinkową**, musimy przesunąć biegun z $S_c$ do $S_s$. Ponieważ profil posiada oś symetrii $OY$, a przesunięcie nastąpiło tylko wzdłuż tej osi (o wartość $\Delta y_s$), wzór transformacyjny upraszcza się do postaci:

$$\omega_{Ss}(s) = \omega_{Sc}(s) - \Delta y_s \cdot z_{glob}(s)$$

Gdzie:
* $\omega_{Sc}(s)$ – funkcja wycinkowa policzona w poprzednim kroku (względem $S_c$),
* $\Delta y_s$ – wyliczona odległość między środkiem ciężkości a środkiem ścinania,
* $z_{glob}(s)$ – współrzędna pozioma (odległość od osi symetrii) danego punktu konturu.

**Interpretacja geometryczna:** Nowa funkcja $\omega_{Ss}$ opisuje spaczenie (deplanację) przekroju przy obrocie wokół rzeczywistego środka obrotu. Jej wartość średnia ważona po przekroju wynosi zero.

####  Wycinkowy moment bezwładności ($I_\omega$)
Parametr ten to miarą odporności przekroju na **spaczenie** (wyboczenie skrętne, deplanację). Im większa wartość $I_\omega$, tym sztywniejszy jest profil przy skręcaniu nieswobodnym.

Wartość ta jest całką z kwadratu głównej współrzędnej wycinkowej po polu przekroju:

$$I_\omega = \int_{A} [\omega_{Ss}(s)]^2 \cdot dA = \sum_{i} \int_{0}^{L_i} [\omega_{Ss, i}(s)]^2 \cdot t_i \cdot ds$$

Obliczenia wykonujemy, sumując całki po wszystkich czteReh strefach zdefiniowanych w modelu (płaskownik, zakładka, środnik, stopka).

#### Biegunowy moment bezwładności ($I_p$)
Do wyznaczenia siły krytycznej przy zwichrzeniu potrzebny jest biegunowy moment bezwładności liczony **względem środka ścinania** ($S_s$), a nie środka ciężkości.

Korzystamy z twierdzenia Steinera. Znamy momenty bezwładności względem osi głównych przechodzących przez środek ciężkości ($I_y$ i $I_z$). Przesuwamy biegun o odległość $r = \Delta y_s$:

$$I_p = I_y + I_z + A_{cal} \cdot (\Delta y_s)^2$$

Promień bezwładności biegunowy ($i_0$):
$$i_0 = \sqrt{\frac{I_p}{A_{cal}}}$$

#### Moment bezwładności przy skręcaniu swobodnym ($I_t$)

Ostatnim parametrem geometrycznym jest moment bezwładności na skręcanie swobodne (Saint-Venanta). Określa on odporność profilu na "czyste" skręcanie, w którym przekrój może się swobodnie paczyć.

Dla profili cienkościennych otwartych obliczamy go jako sumę sztywności poszczególnych ścianek prostokątnych:
$$I_t = \sum \frac{1}{3} b_i t_i^3$$

W naszym profilu sumujemy wkład od:
1.  Płaskownika górnego ($b_p, t_p$)
2.  Dwóch środków ceowników ($c_w, t_{wc}$)
3.  CzteReh stopek ceowników ($c_f, t_{fc}$)

*Uwaga: Wzór ten jest przybliżeniem pomijającym lokalne spiętrzenia naprężeń w narożach i węzłach (tzw. korekta współczynnikiem $\beta$ lub $\alpha$), jednak dla celów projektowych profili składanych jest wystarczający. Dla finalnie wybranej geometrii należy przeliczyć ponownie ten parametr biorąc pod uwagę wartości tabelaryczne współczynników*

#### Podsumowanie jednostek i wielkości fizycznych

W analizie Własowa posługujemy się następującymi wielkościami:

* **Zmienna całkowania ($s$):** $[mm]$ – odległość mierzona wzdłuż konturu profilu.
* **Współrzędna wycinkowa ($\omega$):** $[mm^2]$ – parametr geometryczny określający deplanację (przemieszczenie wzdłuż osi pręta) punktów przekroju. Jest to funkcja zmiennej $s$. Wymagana do skręcania nieswobodnego.
* **Sztywność wycinkowa ($I_\omega$):** $[mm^6]$ – (tzw. stała wycinkowa) miara odporności przekroju na spaczenie. Jest to stała materiałowo-geometryczna (skalar). Wymagana do skręcania nieswobodnego.
* **Biegunowy moment bezwładności ($I_p$):** $[mm^4]$ – miara rozkładu masy względem środka ścinania. Jest to stała geometryczna (skalar).
* **Moment bezwładności przy skręcaniu swobodnym ($I_t$)** - $[mm^4]$ - miara sztywności profilu na skręcanie czyste, swobodne

In [43]:
#GŁÓWNA WSPÓŁRZĘDNA WYCINKOWA, WYCINKOWY MOMENT BEZWŁADNOŚCI, BIEGUNOWY MOMENT BEZWŁADNOŚCI (WZGLĘDEM Ss), MOMENT BEZWŁASNOŚCI PRZY SKRĘCANIU SWOBODNYM

# MOMENT BEZWŁADNOŚCI PRZY SKRĘCANIU SWOBODNYM (It)
# Suma sztywności poszczególnych ścianek (wzór przybliżony dla profili cienkościennych otwartych) It = Suma(1/3 * długość * grubość^3)

# Płaskownik górny
It_plask = (1/3) * bp * tp**3

# Ceowniki (złożone ze środnika i dwóch stopek)
# Korzystamy z wcześniej policzonych długości obliczeniowych cw - wysokość płaska środnika i cf - wysięg płaski stopki. Dodatkowo uwzględniamy naroża (uproszczone podejście, sumowania prostokątów). Dla dokładniejszych wyników można brać It z katalogu
It_ceownik_pojedynczy = (1/3) * (cw * twc**3 + 2 * (cf * tfc**3))

# Całkowity moment skręcania swobodnego
It = It_plask + 2 * It_ceownik_pojedynczy

# 3. OBLICZENIE I_OMEGA (Iw) oraz I_BIEGUNOWEGO (Ip)

# BEZPIECZNIK OBLICZEŃ:
# True  -> Próbuje wykonać pełne całkowanie symboliczne.
# False -> Tworzy symbole zamiast wyników, aby umożliwić dalszą pracę bez podstawiania wartości.
WYKONAJ_PELNE_CALKOWANIE = False

if WYKONAJ_PELNE_CALKOWANIE:
    print("ROZPOCZĘTO PEŁNE CAŁKOWANIE")
    # Całkowanie kwadratów funkcji omega_ss
    Iw_1 = sp.integrate(omega_ss_1**2 * tp, (s_var, 0, dlugosc_strefy_1))
    Iw_2 = sp.integrate(omega_ss_2**2 * t_zakladka, (s_var, 0, dlugosc_strefy_2))
    Iw_3 = sp.integrate(omega_ss_3**2 * twc, (s_var, 0, dlugosc_strefy_3))
    Iw_4 = sp.integrate(omega_ss_4**2 * tfc, (s_var, 0, dlugosc_strefy_4))

    # Suma dla całego profilu (x2 - symetria)
    Iw = 2 * (Iw_1 + Iw_2 + Iw_3 + Iw_4)
   
    # Biegunowy moment bezwładności (Steiner od Sc do Ss)
    Ip = Iy + Izc + Acal * delta_ys**2
    
    # Promień bezwładności biegunowy
    io = sp.sqrt(Ip / Acal)
    
else:
    print("TRYB PROJEKTOWY: Pominięto całkowanie Iw. Zmienne to symbole.")
    
    # Deklaracja symboli, aby solver nie wyrzucił błędu
    Iw = sp.symbols('Iw', real=True, positive=True)
    Ip = sp.symbols('Ip', real=True, positive=True)
    io = sp.symbols('i_0', real=True, positive=True)

TRYB PROJEKTOWY: Pominięto całkowanie Iw. Zmienne to symbole.


## Kompleksowa Analiza Wytrzymałości i Stateczności Słupa Podnośnika
### Model Pręta Cienkościennego wg Teorii Własowa i Normy PN-EN 1993-1-1

Niniejszy dokument przedstawia szczegółową procedurę obliczeniową dla asymetrycznego słupa podnośnika samochodowego. Analiza wykracza poza elementarną wytrzymałość materiałów, uwzględniając złożony stan obciążenia (ściskanie mimośrodowe, zginanie dwukierunkowe, skręcanie nieswobodne) oraz nieliniowe zjawiska utraty stateczności (wyboczenie giętno-skrętne), zwichrzenie.

#### 1. Zdefiniowanie Modelu Fizycznego i Schematu Statycznego

Prawidłowe odwzorowanie rzeczywistości inżynierskiej wymaga przyjęcia odpowiednich założeń brzegowych:

1.  **Schemat Statyczny: Wspornik**
    Słup podnośnika jest elementem utwierdzonym w fundamencie (zablokowane wszystkie translacje i rotacje u dołu) oraz swobodnym na wierzchołku. Przyjmujemy, konstrukcje słupa przyspawaną do podstawy.
    * Implikacja dla momentów: Maksymalne momenty zginające występują w utwierdzeniu.
    * Implikacja dla bimomentu: W utwierdzeniu deplanacja jest zablokowana ($\phi' = 0$), co generuje maksymalny bimoment. Na końcu swobodnym deplanacja jest możliwa, bimoment wynosi zero.

2.  **Długość Wyboczeniowa ($L_{cr}$)**
    Dla idealnego wspornika teoretyczny współczynnik długości wyboczeniowej wynosi $\mu = 2.0$.
    $$L_{cr} = 2 \cdot L_{belka}$$
    Oznacza to, że słup zachowuje się pod względem stateczności tak, jakby był elementem wolnopodpartym o dwukrotnie większej długości. Jest to założenie kluczowe i bezpieczne.

3.  **Charakterystyka Przekroju: Cienkościenny Otwarty**
    Profil złożony z ceownika i płaskownika jest przekrojem **monosymetrycznym** (oś symetrii OY).
    * Środek Ciężkości ($S_c$) i Środek Ścinania ($S_s$) nie pokrywają się.
    * Istnieje mimośród geometryczny $y_0 = \Delta y_s$, który powoduje mechaniczne sprzężenie zginania ze skręcaniem.

#### 2. Szczegółowa Analiza Obciążeń i Redukcja Sił

W kodzie dokonujemy redukcji wszystkich sił zewnętrznych do osi pręta, uwzględniając ich wpływ na poszczególne stopnie swobody.

##### A. Wpływ Siły Osiowej ($F_x$) - Ściskanie i Zginanie Mimośrodowe
Siła ciężkości (pojazd + ramię) oznaczona jako $F_x$ działa na ramieniu podnośnika, a nie w osi słupa.
* **Efekt 1 (Ściskanie):** Bezpośrednia siła ściskająca $N = F_x$.
* **Efekt 2 (Moment od mimośrodu):** Siła ta działa na ramieniu $e = F_{promien} - y_c$. Generuje to stały na wysokości słupa moment zginający $M_{z, fix} = F_x \cdot e$.

##### B. Wpływ Sił Poprzecznych, równoległych do osi głównych przekroju i oddalonych o to samo ramie $F_{promien}$ ($T_y$ / $F_y$)
Siła działająca prostopadle do słupa wzdłuż osi symietrii ($OY$) (np. od nierównomiernego obciążenia, sił dynamicznych, nachylenia podłoża itp).
* **Zginanie pasma względem OZ ($M_z$):** Siła ta działa na ramieniu równym długości słupa, licząc od miejsca jego utwierdzenia. Dla wspornika moment rośnie liniowo od zera (góra) do maksimum (dół): $M_{z, var} = T_y \cdot L$.
* **Ścinanie działające w osi przekroju ($T_y$):**

**Superpozycja Momentów $M_z$:**
Sumaryczny moment zginający względem osi mocnej to suma modułów obu powyższych oddziaływań (najgorszy przypadek):
$$M_{z,Ed} = |F_x \cdot (F_{promien} - y_c)| + |T_y \cdot L|$$
Sytuacja ta w rzeczywistości nie powinna mieć miejsca, ponieważ sugeruje takie podparcie ładunku, że słup jest dociągany do ładunku, a nie odpychany od niego.

##### C. Wpływ Siły Poprzecznej z płaszczyzny ($T_z$ / $F_z$)
Siła boczna (od nierównomiernego obciążenia, uderzenia, siły bezwładności, czy podmuchu wiatru). Powoduje najbardziej złożony stan naprężenia złożony z:
* **Zginanie boczne ($M_y$):** Wygięcie słupa w "słabszą" stronę. $M_{y,Ed} = T_z \cdot L$.
* **Moment Skręcający ($M_s$):** Ponieważ siła działa na ramieniu, nie przechodzi przez Środek Ścinania ($S_s$). Generuje moment skręcający:
    $$M_s = T_z \cdot (F_{promien} + y_s)$$
* **Siła tnąca działająca w kierunku OZ**

#### 3. Teoria Własowa: Skręcanie Nieswobodne i Bimoment

Profile otwarte (jak ceowniki) są bardzo wiotkie na skręcanie. Gdy próbujemy je skręcić, ich przekroje nie pozostają płaskie – ulegają **deplanacji** (wyboczeniu paczenia, spaczeniu, ang. *warping*).
Gdy zablokujemy tę deplanację (wspawanie słupa w podstawę), powstaje system sił wewnętrznych zwany **Bimomentem ($B_\omega$)**.

* Bimoment to "moment par sił", mierzony w $[N \cdot mm^2]$.
* Wywołuje on naprężenia normalne ($\sigma$), które dodają się do naprężeń od zginania i ściskania.
* Wzór analityczny dla wspornika obciążonego momentem skupionym na końcu:
    $$B(x) = \frac{M_s}{k} \tanh(kL)$$
    Gdzie parametr $k = \sqrt{\frac{G I_t}{E I_\omega}}$ określa, jak szybko zanika wpływ utwierdzenia w głąb belki.

#### 4. Weryfikacja Stanu Naprężenia (SGU/SGN)

Dla każdego zdefiniowanego punktu przekroju ($P1 \dots P6$) sprawdzamy wytężenie materiału. Punkty to:
* $P1$ - środek płaskownika, punkt na osi symetrii 
* $P2$ - punkt łączenia płaskownika z ceownikiem - powstaje zakładka, zmiana grubości i zmiana położenia y środka ciężkości
* $P3$ - naroże płaskownika 
* $P4$ - punkt na środniku ceownika na wysokości środka cieżkości
* $P5$ - punkt w dolnym narożu ceownika
* $P6$ - punkt na końcu półki ceownika, wolny koniec, przekrój otwarty

##### KROK 1 Naprężenia normalne - Oś wzdłużna
Sumujemy cztery niezależne składowe:
1.  **Ściskanie:** $\sigma_N = \frac{N}{A}$ (rozkład równomierny).
2.  **Zginanie Główne ($M_z$):** $\sigma_{Mz} = \frac{M_z}{I_z} \cdot y$ (ściska jedną półkę, rozciąga drugą).
3.  **Zginanie Boczne ($M_y$):** $\sigma_{My} = \frac{M_y}{I_y} \cdot z$ (ściska "przód", rozciąga "tył").
4.  **Spaczenie ($B_\omega$):** $\sigma_{B} = \frac{B_\omega}{I_\omega} \cdot \omega(s)$.
    * Funkcja $\omega(s)$ przyjmuje wartości ekstremalne na końcach stopek. To dlatego bimoment jest groźny dla profili otwartych.

Wartość maksymalną sumy uzykujemy poprzez sprawdzenie w pętli każdego z warunków dla wszystkich zdefiniowanych powyżej punktów.

##### Krok 2: Naprężenia Styczne ($\tau$)
Sumujemy ścinanie i skręcanie:
1.  **Ścinanie siłą $T_y$:** $\tau_{Vy} = \frac{T_y S_z}{I_z \cdot t}$ (wzór Żurawskiego).
2.  **Ścinanie siłą $T_z$:** $\tau_{Vz} = \frac{T_z S_y}{I_y \cdot t}$.
3.  **Skręcanie swobodne:** $\tau_t = \frac{M_s \cdot t}{I_t}$ (cyrkulacja naprężeń po grubości ścianki).

##### Krok 3: Hipoteza Wytężeniowa (Von Mises)
$$\sigma_{zast} = \sqrt{\sigma_{tot}^2 + 3 \tau_{tot}^2} \le f_y$$
Otrzymujemy naprężenia zredukowane, do przyrównania do właściwości materiałowych pod kątem przekroczenia granicy plastyczności i przyjętych współczynników bezpieczeństwa.

#### 5. Analiza Stateczności: Siły Krytyczne i Interakcja

Jest to najważniejszy punkt weryfikacji słupów smukłych. Musimy sprawdzić, przy jakiej sile słup "ucieknie" z położenia równowagi. W tym słupie mogą potencjalnie wystąpić następujące utraty stateczności wywołane ściskaniem siłą osiową: wyboczenie gięte względem OY, wyboczenie gięte względem OZ, wyboczenie skrętne, wyboczenie gięto-skrętne praz zwichrzenie wywołane momentem gnącym (zginaniem mimośrodowym).

##### Dlaczego wyboczenie giętno-skrętne?
Ponieważ $S_c \neq S_s$, oś obrotu profilu nie pokrywa się z osią masy.
* Gdy słup wygina się względem osi mocnej ($Z$), środek ścinania przemieszcza się na bok, co indukuje moment skręcający.
* Gdy słup się skręca, środek ciężkości przemieszcza się na bok, co indukuje moment zginający.
To sprzężenie zwrotne sprawia, że siła krytyczna dla układu ($N_{cr, TF}$) jest **zawsze mniejsza** niż siła czystego wyboczenia ($N_{cr, z}$) i czystego skręcania ($N_{cr, T}$).

**Algorytm obliczeń w kodzie:**
1.  Obliczamy $N_{cr, y}$ (Euler dla osi słabej) – postać niezależna.
2.  Obliczamy $N_{cr, z}$ (Euler dla osi mocnej) i $N_{cr, s}$ (Skrętne) – składniki postaci sprzężonej.
3.  Rozwiązujemy równanie kwadratowe interakcji (wg Timoszenki):
    $$r_0^2 (N - N_z)(N - N_s) - N^2 y_0^2 = 0$$
    Gdzie $y_0$ to mimośród $S_c - S_s$.
4.  Wybieramy minimum globalne: $N_{cr, min} = \min(N_{cr, y}, N_{cr, gs})$.

#### 6. Implementacja Normowa (PN-EN 1993-1-1)

Przejście z teorii sprężystej (Euler) na nośność inżynierską.

1.  **Smukłość Względna ($\bar{\lambda}$):**
    Porównuje nośność plastyczną przekroju ($A \cdot R_e$) z siłą krytyczną ($N_{cr}$).
    $$\bar{\lambda} = \sqrt{\frac{A \cdot R_e}{N_{cr}}}$$

2.  **Krzywa Wyboczeniowa i Parametr Imperfekcji ($\alpha$):**
    Norma zakłada, że rzeczywisty słup ma wstępne wygięcia i naprężenia spawalnicze.
    * W kodzie przyjęto krzywą **'c'** ($\alpha = 0.49$).
    * Jest to wartość zalecana dla profili spawanych (UPE + Płaskownik), które są bardziej podatne na wyboczenie niż profile walcowane.

3.  **Współczynnik Redukcji ($\chi$):**
    Parametr redukujący nośność materiału ($R_e$) ze względu na ryzyko wyboczenia.
    $$\chi = \frac{1}{\Phi + \sqrt{\Phi^2 - \bar{\lambda}^2}}$$
    Obliczany automatycznie przez funkcję `oblicz_wspolczynnik_chi`.

---

#### 7. Warunek Końcowy (UR - Wytężenie, stopień wykorzystania nośności)

Ostatnim krokiem jest sprawdzenie warunku interakcji wszystkich sił wewnętrznych, zgodnie z zasadą sumowania wytężeń:

$$UR = \frac{N_{Ed}}{N_{b,Rd}} + \frac{M_{y,Ed}}{M_{b,Rd}} + \frac{M_{z,Ed}}{M_{c,Rd}} + \frac{B_{Ed}}{M_{\omega, Rd}} \le 1.0$$

* Składnik 1: Ściskanie z uwzględnieniem wyboczenia.
* Składnik 2: Zginanie główne z uwzględnieniem zwichrzenia ($M_{cr}$).
* Składnik 3: Zginanie boczne (zazwyczaj bez zwichrzenia dla osi słabej).
* Składnik 4: Spaczenie (Bimoment) – weryfikacja naprężeń w najdalszym narożu (P6).

Jeśli $UR \le 1.0$, konstrukcja jest bezpieczna wg przyjętych norm i współczynników bezpieczeństwa.

In [45]:
# WARUNKI BRZEGOWE, OBCIĄŻENIA, STAŁE MATERIAŁOWE I WSPÓŁCZYNNIKI BEZPIECZEŃSTWA

# Stałe materiałowe (Symbole - do podstawienia np. Stal S355)
E = sp.symbols('E', real=True, positive=True)      # Moduł Younga [MPa]
G = sp.symbols('G', real=True, positive=True)      # Moduł Kirchhoffa [MPa]
#Re -zadeklarowane wcześniej, Granica plastyczności [MPa]

# Współczynniki bezpieczeństwa i paramtery obciążenia
x_ogolne = sp.symbols('x_ogolne', real=True, positive=True)  # Wytrzymałość ogólna (np. 2.0)
x_statecznosc = sp.symbols('x_statecznosc', real=True, positive=True) # Stateczność (np. 4.0)
w_Ty, w_Tz=sp.symbols('w_Ty w_Tz', real=True, positive=True)

# DEFINICJA OBCIĄŻENIA I GEOMETRII BELKI

# Wektory siły zewnętrznej i geometrii belki
F_x = sp.symbols('F_x', real=True) # Siła osiowa [N]
F_promien = sp.symbols('F_promien', real=True)    # Punkt przyłożenia sił (wsp. Y od góry profilu)
L_belka = sp.symbols('L_belka', real=True, positive=True) # Długość FIZYCZNA słupa [mm]

#REDUKCJA SIŁ DO ŚRODKÓW CIĘŻKOŚCI I ŚCINANIA (SIŁY WEWNĘTRZNE)
F_N=F_x #siła osiowa
T_y=F_x*w_Ty #siła poprzeczna - Tnąca i zginająca
T_z=F_x*w_Tz #siła poprzeczna - Skręcająca i zginająca

# MOMENT ZGINAJĄCY WZGLĘDEM OSI Z (Mgz)
# 1. Od mimośrodu siły Fx: M = Fx * ramię. Ramię to (F_promien - yc). UWAGA ramie liczone od osi płaskownika
Mgz_fx = F_x * (F_promien - yc)
# 2. Od siły poprzecznej Fy: M = Fy * L (Dla wspornika moment max jest w utwierdzeniu)
Mgz_fy = T_y * L_belka
# Superpozycja (Suma modułów momentó gnących da najgorszy przypadek)
Mgz = sp.Abs(Mgz_fx) + sp.Abs(Mgz_fy)

# MOMENT ZGINAJĄCY WZGLĘDEM OSI Y (Mgy) zginanie "lewo-prawo" (boczne) od siły boczne Fz
# Dla wspornika: M = F * L
Mgy = T_z * L_belka

# MOMENT SKRĘCAJĄCY (Ms) moment skręcający liczony względem Środka Ścinania (Ss).
# Ramię = F_promien + ys (gdzie ys to odległość Ss od OZ).
Ms = T_z * (F_promien + ys)

# Bimoment (B_w) - Parametr spaczenia dla przekroju. Warunki brzegowe: na końcu swobodnym Bimoment = 0 (swobodna deplanacja), w utwierdzenie deplanacja zablokowana -> Bimoment MAKSYMALNY.
# Parametr sztywności skrętnej k (charakterystyka przekroju):
k_skret = sp.sqrt((G * It) / (E * Iw))
# Wzór analityczny na B_max dla wspornika obciążonego momentem skupionym na końcu B_max = (T / k) * tanh(k * L)
B_w = (T_z/ k_skret) * sp.tanh(k_skret * L_belka)

# OBLICZENIE MOMENTÓW STATYCZNYCH (Sz, Sy) DO NAPRĘŻEŃ STYCZNYCH
# Aby policzyć Tau (wzór Żurawskiego), musimy znać moment statyczny odciętej części przekroju. Całkujemy y*t*ds (dla Sz) oraz z*t*ds (dla Sy) po konturze. Zmienne pomocnicze (s_var zdefiniowane wcześniej)

# ETAP 1: PŁASKOWNIK
# y_loc = -yc (stałe), z_loc = s_var
Sz_1 = sp.integrate(-yc * tp, (s_var, 0, s_var))
Sy_1 = sp.integrate(s_var * tp, (s_var, 0, s_var))
Sz_koniec_1 = Sz_1.subs(s_var, dlugosc_strefy_1)
Sy_koniec_1 = Sy_1.subs(s_var, dlugosc_strefy_1)

# ETAP 2: ZAKŁADKA
y_zakl_glob = (tp * 0 + tfc * (tp/2 + tfc/2)) / (tp + tfc) #współrzędna y środka ciężkości zakładki w układzie globalnym - średnia ważona
y_zakl_loc = y_zakl_glob - yc # współrzędna y lokalna zakładki
# z_loc = dlugosc_strefy_1 + s_var
Sz_2 = Sz_koniec_1 + sp.integrate(y_zakl_loc * (tp + tfc), (s_var, 0, s_var))
Sy_2 = Sy_koniec_1 + sp.integrate((dlugosc_strefy_1 + s_var) * (tp + tfc), (s_var, 0, s_var))
Sz_koniec_2 = Sz_2.subs(s_var, dlugosc_strefy_2)
Sy_koniec_2 = Sy_2.subs(s_var, dlugosc_strefy_2)

# ETAP 3: ŚRODNIK
# y_loc = s_var - yc (idziemy w dół od 0), z_loc = z_c (stałe)
Sz_3 = Sz_koniec_2 + sp.integrate((s_var - yc) * twc, (s_var, 0, s_var))
Sy_3 = Sy_koniec_2 + sp.integrate(z_c * twc, (s_var, 0, s_var))
Sz_koniec_3 = Sz_3.subs(s_var, dlugosc_strefy_3)
Sy_koniec_3 = Sy_3.subs(s_var, dlugosc_strefy_3)

# ETAP 4: STOPKA DOLNA
# y_loc = y_f - yc (stałe), z_loc = z_c - s_var
Sz_4 = Sz_koniec_3 + sp.integrate((y_f - yc) * tfc, (s_var, 0, s_var))
Sy_4 = Sy_koniec_3 + sp.integrate((z_c - s_var) * tfc, (s_var, 0, s_var))

# TABELA PUNKTÓW KRYTYCZNYCH 

punkty_definicja = {
    "P1": {
        "opis": "Środek płaskownika",
        "y_glob": 0, "z_glob": 0, "t_scianki": tp,
        # Omega względem Ss (bezpośrednio z funkcji)
        "omega_Ss": omega_ss_1.subs(s_var, 0),
        "Sz": Sz_1.subs(s_var, 0), "Sy": Sy_1.subs(s_var, 0)
    },
    "P2": {
        "opis": "Koniec nakładki (płaskownik)",
        "y_glob": 0, "z_glob": dlugosc_strefy_1, "t_scianki": tp,
        # Koniec strefy 1
        "omega_Ss": omega_ss_1.subs(s_var, dlugosc_strefy_1),
        "Sz": Sz_koniec_1, "Sy": Sy_koniec_1
    },
    "P3": {
        "opis": "Górne naroże (oś środnika)",
        "y_glob": 0, "z_glob": z_c, "t_scianki": twc,
        # Koniec strefy 2
        "omega_Ss": omega_ss_2.subs(s_var, dlugosc_strefy_2),
        "Sz": Sz_koniec_2, "Sy": Sy_koniec_2
    },
    "P4": {
        "opis": "Środnik na wysokości osi obojętnej (Yc)",
        "y_glob": yc, "z_glob": z_c, "t_scianki": twc,
        # Strefa 3 w punkcie yc
        "omega_Ss": omega_ss_3.subs(s_var, yc),
        "Sz": Sz_3.subs(s_var, yc), "Sy": Sy_3.subs(s_var, yc)
    },
    "P5": {
        "opis": "Dolne naroże",
        "y_glob": y_f, "z_glob": z_c, "t_scianki": tfc,
        # Koniec strefy 3
        "omega_Ss": omega_ss_3.subs(s_var, dlugosc_strefy_3),
        "Sz": Sz_koniec_3, "Sy": Sy_koniec_3
    },
    "P6": {
        "opis": "Koniec dolnej półki",
        "y_glob": y_f, "z_glob": z_c - z_f, "t_scianki": tfc,
        # Koniec strefy 4
        "omega_Ss": omega_ss_4.subs(s_var, dlugosc_strefy_4),
        "Sz": Sz_4.subs(s_var, dlugosc_strefy_4), "Sy": Sy_4.subs(s_var, dlugosc_strefy_4)
    }
}

lista_sigma_vm = []

for k, d in punkty_definicja.items():
    # Współrzędne lokalne (do wzorów Naviera - zginanie)
    # y_loc: odległość od osi obojętnej Zc (przechodzącej przez Sc)
    y_loc = d["y_glob"] - yc
    # z_loc: odległość od osi obojętnej Y (osi symetrii)
    z_loc = d["z_glob"]
    
    # Omega względem Ss (do Bimomentu - spaczenie) - POBIERAMY GOTOWĄ WARTOŚĆ Z TABELI (policzoną z omega_ss_i)
    omega_Ss_pkt = d["omega_Ss"]
    
    # Naprężenia Normalne (Sigma X)
    sig_N = F_N / Acal
    sig_Mz = (Mgz / Izc) * y_loc
    sig_My = (Mgy / Iy) * z_loc
    sig_B = (B_w / Iw) * omega_Ss_pkt
    
    # Suma naprężeń normalnych
    sigma_x_total = sig_N + sig_Mz + sig_My + sig_B
    
    # Naprężenia Styczne (Tau)
    # Wzór Żurawskiego (V*S / I*t) + Skręcanie swobodne (T*t / It)
    tau_Vy = (T_y * d["Sz"]) / (Izc * d["t_scianki"])
    tau_Vz = (T_z * d["Sy"]) / (Iy * d["t_scianki"])
    # Uwaga: Dla profili otwartych tau_max od skręcania swobodnego = Ms * t / It
    tau_T = (Ms * d["t_scianki"]) / It 
    
    # Sumujemy moduły (Najgorszy możliwy przypadek zwrotów sił)
    tau_total = sp.Abs(tau_Vy) + sp.Abs(tau_Vz) + sp.Abs(tau_T)
    
    # 5. Hipoteza Von Misesa
    sigma_vm = sp.sqrt(sigma_x_total**2 + 3 * tau_total**2)
    lista_sigma_vm.append(sigma_vm)


# 6. WARUNEK 2: STATECZNOŚĆ (SIŁY KRYTYCZNE I INTERAKCJA)
# POPRAWKA DLA WSPORNIKA: Długość wyboczeniowa L_cr = 2 * L_belka

# Siły Krytyczne
# 1. Wyboczenie Giętne (Euler względem OY, zginanie od F_z)
# Dla wspornika: L_cr = 2 * L_belka
N_cr_gy = (sp.pi**2 * E * Iy) / (2 * L_belka)**2

# 2. Wyboczenie Giętne (Euler względem OZ, zginanie do F_y)
# Dla wspornika: L_cr = 2 * L_belka
N_cr_gz = (sp.pi**2 * E * Izc) / (2 * L_belka)**2

# 3. Wyboczenie Skrętne (Czyste)
# Korzystamy z io (promień bezwładności biegunowy) i Iw
# Dla skręcania wspornikowego (utwierdzenie-swobodny) człon z Iw przyjmuje długość wyboczeniową 2*L
N_cr_s = (1 / io**2) * (G * It + (sp.pi**2 * E * Iw) / (2 * L_belka)**2)

## Wyboczenie Giętno-Skrętne  - N_cr_gs
# Podstawa teoretyczna: Rozwiązanie równania charakterystycznego Timoszenki.
# Równanie wyjściowe: (N - N_z)*(N - N_T)*i0^2 - N^2*y0^2 = 0
# Postać po wymnożeniu i przekształćeniach: N^2 * [1 - (y0/i0)^2] - N * (N_z + N_T) + (N_z * N_T) = 0
# Postać równania kwadratowego: A*N^2 + B*N + C = 0

# Zmienna pomocnicza - parametr beta
y0 = delta_ys  # Mimośród (odległość Sc-Ss)
r0 = io        # Promień bezwładności biegunowy względem Ss

# Współczynnik A (To nasz nawias kwadratowy - parametr Beta):
beta_param = 1 - (y0 / r0)**2
param_A = beta_param
# Współczynnik B (To co stoi przy N - suma sił z minusem):
param_B = -(N_cr_gz + N_cr_s)
# Współczynnik C (Wyraz wolny - iloczyn sił):
param_C = N_cr_gz * N_cr_s

# Obliczenie Delty i Rozwiązanie
# Delta = B^2 - 4*A*C
delta_rownania = param_B**2 - 4 * param_A * param_C
# Równanie kwadratowe ma dwa rozwiązania matematyczne. Fizycznie oba opisują stany równowagi, ale utrata stateczności nastąpi przy sile najmniejszej (pierwszej napotkanej przy obciążaniu).
# Rozwiązanie 1 (z minusem przy delcie):
N_root_1 = (-param_B - sp.sqrt(delta_rownania)) / (2 * param_A)
# Rozwiązanie 2 (z plusem przy delcie):
N_root_2 = (-param_B + sp.sqrt(delta_rownania)) / (2 * param_A)

# Wybieramy mniejszą wartość (funkcja Min z pakietu SymPy):
N_cr_gs = sp.Min(N_root_1, N_root_2)

# Moment Krytyczny Zwichrzenia (M_cr)
# Dla wspornika w uproszczonym podejściu stosujemy długość wyboczeniową L_cr = 2*L w członie wyboczeniowym
M_cr = (sp.pi**2 * E * Iy) / ((2 * L_belka)**2) * sp.sqrt((Iw / Iy) + ((2 * L_belka)**2 * G * It) / (sp.pi**2 * E * Iy))

# Wybór miarodajnej siły krytycznej (min z GY i GS)
N_cr_min = sp.Min(N_cr_gy, N_cr_gz, N_cr_gs)

# OBLICZENIE WSPÓŁCZYNNIKÓW WYBOCZENIOWYCH (CHI) WG PN-EN 1993-1-1

# Parametr imperfekcji (alfa)
# Określa wrażliwość profilu na niedoskonałości geometryczne i naprężenia własne. Wartości wg Tabeli 6.1 (PN-EN 1993-1-1):
# alfa=0.13 -> (Krzywa a0) Stal wysokiej jakości, profile gorącowalcowane
# alfa=0.21 -> (Krzywa a)  Profile walcowane (np. dwuteowniki IPE)
# alfa=0.34 -> (Krzywa b)  Profile o grubszych ściankach
# alfa=0.49 -> (Krzywa c)  Profile spawane, ceowniki
# alfa=0.76 -> (Krzywa d)  Profile bardzo grube lub spawy o dużej ingerencji

alfa_imperfekcji = 0.49  # Przyjęto dla profilu spawanego (krzywa c)

# Funkcja obliczająca współczynnik redukcji Chi (wzór 6.49)
def oblicz_wspolczynnik_chi(lambda_rel, alfa):
    """Oblicza współczynnik wyboczeniowy Chi na podstawie smukłości i imperfekcji.
    Realizuje wzór: Chi = 1 / (Phi + sqrt(Phi^2 - lambda^2))"""
    # Parametr pomocniczy Phi (wzór 6.53) Phi = 0.5 * [1 + alfa * (lambda - 0.2) + lambda^2]
    Phi = 0.5 * (1 + alfa * (lambda_rel - 0.2) + lambda_rel**2)
    # Współczynnik redukcji Chi
    chi = 1 / (Phi + sp.sqrt(Phi**2 - lambda_rel**2))
    # Zabezpieczenie normowe: Chi <= 1.0 (nie może zwiększać nośności)
    return sp.Min(1.0, chi)

# OBLICZENIA WŁAŚCIWE 
# Smukłości względne (Lambdy) lambda = pierwiastek(Afy / Ncr), używamy N_cr_min (decydującej siły krytycznej z analizy giętno-skrętnej)
lambda_N = sp.sqrt(Acal * Re / N_cr_min) 
lambda_LT = sp.sqrt(Wz * Re / M_cr)      

# 2. Obliczenie współczynników Chi z podstawionym parametrem alfa (0.49)
chi_N = oblicz_wspolczynnik_chi(lambda_N, alfa_imperfekcji)
chi_LT = oblicz_wspolczynnik_chi(lambda_LT, alfa_imperfekcji)

# Nośności obliczeniowe
N_Rd_stab = (chi_N * Acal * Re) / x_statecznosc
Mz_Rd_stab = (chi_LT * Wz * Re) / x_statecznosc

# Do nośności na bimoment bierzemy omega z punktu P6 (koniec półki - tam max spaczenie)
omega_max_stat = sp.Abs(punkty_definicja["P6"]["omega_Ss"])
Mw_Rd_stab = (Iw / omega_max_stat) * (Re / x_ogolne) 

# FINALNY WARUNEK (UR <= 1.0)
UR_stability = (
    (sp.Abs(F_N) / N_Rd_stab) +              
    (sp.Abs(Mgz) / Mz_Rd_stab) +            
    (sp.Abs(Mgy) / ((Wy * Re)/x_ogolne)) + 
    (sp.Abs(B_w) / Mw_Rd_stab)                
)